In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
import requests
from urllib import request, parse

from selenium.webdriver.chrome.options import Options
from selenium import webdriver

In [89]:
def find_regular_expression(string_tofind, pattern, group):
    string_pattern = re.compile(pattern)
    result = re.match(string_pattern, string_tofind)
    if result is None:
        return ''
    else:
        return result.group(group)

def find_salary(salaries, unit=False, min_value=True):
    salary_pattern = '^([0-9]+)-([0-9]+)([万千kK]+)'
    position = 0
    if unit == True:
        position = 3
    else:
        if min_value == True:
            position = 1
        else:
            position = 2
    return find_regular_expression(salaries, salary_pattern, position)

def find_javascript_content(javascript, text):
    return find_regular_expression(javascript, ".*"+text+":'([^']+)'.*", 1)

In [ ]:
prefix = "https://www.zhipin.com"
output_file = "C:\\Users\\hao.ren3\\Desktop\\list_boss.csv"
list_column = ['job_id', 'job_name', 'job_salary_min', 'job_salary_max', 'job_salary_unit', 'company','update_date', 'url', 'work_place']
initialization = pd.DataFrame(columns=list_column)
initialization.to_csv(output_file, columns=list_column, index=None)
# url = "https://www.zhipin.com/job_detail/39a12d90e306655903d83ti7FFA~.html"
url = "https://www.zhipin.com/c101270100-p150301/?page=1&ka=page-1"
prefix = "https://www.zhipin.com"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'cookie': 'lastCity=101270100; _uab_collina=156767462968203235331778; __c=1567992279; __g=-; Hm_lvt_194df3105ad7148dcf2b98a91b5e727a=1567732354,1567737011,1567992279,1567992285; Hm_lpvt_194df3105ad7148dcf2b98a91b5e727a=1567992285; __l=l=%2Fwww.zhipin.com%2F&r=https%3A%2F%2Fwww.google.com%2F&friend_source=0&friend_source=0; __a=1197973.1567674630.1567737011.1567992279.34.4.3.34; __zp_stoken__=e62234orkCh5IxdgwoDIYNpmfjTXlhc2yoPpr7RgPL3sFLwO08lIwRBy9QzlVa9uwQrKS1IblqyBlmSW9hjNkUEhBQ%3D%3D'
}

while(True):
    req = request.Request(url=url, headers=headers, method='GET')
    response = request.urlopen(req)
    sub_bs_list = BeautifulSoup(response.read().decode('utf-8'), 'html.parser')

    list_job_link = []
    data_index = re.compile("^[0-9]+$")
    original_data = sub_bs_list.find_all('div', {'class': 'job-primary'})
    for i in original_data:
        list_job_link.append(prefix+ "" + i.find_all('a', {'data-index': data_index})[0].attrs['href'])
    
    for current_job_link in list_job_link:
        req = request.Request(url=current_job_link, headers=headers, method='GET')
        response = request.urlopen(req)
        sub_bs = BeautifulSoup(response.read().decode('utf-8'), 'html.parser')
        javascript = sub_bs.find_all('script', {'type':'text/javascript'})[0].get_text()
        javascript = javascript.replace("\r", "").replace("\n", "").replace(" ", "")
        job_name = find_javascript_content(javascript, "job_name")
        reportedId = find_javascript_content(javascript, "reportedId")
        targetId = find_javascript_content(javascript, "targetId")
        job_id = find_javascript_content(javascript, "job_id")
        job_salary = find_javascript_content(javascript, "job_salary")
        job_salary_min = find_salary(job_salary)
        job_salary_max = find_salary(job_salary, min_value=False)
        job_salary_unit = find_salary(job_salary, unit=True)
        company = find_javascript_content(javascript, "company")

        update_date = sub_bs.find_all('p', {'class':'gray'}, text=re.compile('.*更新于：.*'))[0].get_text().replace("更新于：", "")
        place = sub_bs.find_all('p')
        pattern = re.compile("<[^<]+</em>")
        result_list = [job_id, job_name, job_salary_min, job_salary_max, job_salary_unit, company, update_date, url]

        for i in place:
            if  '<em class="dolt"' in str(i):
                result = i.get_text()
                test = str(i).split('<em class="dolt"></em>')
                for j in test:
                    result_list.append(j.replace("<p>", "").replace("</p>", ""))
        result_list = pd.DataFrame(np.reshape(result_list, (1, -1)))
        result_list.to_csv(output_file, index=False, header=False, mode='a')
        time.sleep(5)
    next_page = sub_bs_list.find_all('a', {'ka':'page-next'})
    if len(next_page)>0:
        url = prefix + "" + sub_bs_list.find_all('a', {'ka':'page-next'})[0].attrs['href']
    else:
        break